In [1]:
from src.infer_models import infer
from src.preprocess import preprocess
from src.archs_models import stdlogistic
from src.opts_models import batchgd
import os
import sys
import pathlib
from pathlib import Path

import numpy as np
from numpy.core.shape_base import block
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Computer Modern Sans Serif"]})  # Avant Garde, Helvetica, Computer Modern Sans Serif
# for Palatino and other serif fonts use:
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "serif",
#     "font.serif": ["Computer Modern Roman"], # Times, Bookman, Pa;atino
# })
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "monospace",
#     "font.monospace": ["Consolas"],
# })

np.set_printoptions(precision=4)
np.set_printoptions(formatter={'float': "{:0.4f}".format})


# Ensure path is referenced to this script's root
# thisdir = os.path.dirname(__file__)
thisdir = Path.cwd()
# os.chdir(thisdir)
os.chdir(sys.path[0])
print(os.getcwd())

figs_dir = os.path.join(thisdir, 'figs/')
if not os.path.isdir(figs_dir):
    os.makedirs(figs_dir)

# os.chdir(r'./ai534ias/ia1/')

# Generate the path to the file relative to your python script:
# script_location = Path(__file__).absolute().parent
# print(script_location)
# file_location = script_location / 'file.yaml'
# file = file_location.open()


/home/somefunagba/ias/ia2


In [2]:
# Data Preprocessing

# do major feature engineering - 0 | 1
doengr = 1
donormalize = 1

# Full Train
rawdata = 'kaggle/IA2-train.csv'
dataframet = pd.read_csv(rawdata)
rawdata = 'kaggle/IA2-dev.csv'
dataframev = pd.read_csv(rawdata)
dataframe_combo = pd.concat([dataframet, dataframev])
dataframe_combo.to_csv('kaggle/IA2_combo.csv', index=False)
rawdata = 'kaggle/IA2_combo.csv'
traindata, train_id = preprocess(rawdata, donormalize=donormalize, istrain=1,
                                 traininfo=None, doengr=doengr)


# View final data entering the model.
# print(traindata['X'])

# Dev
rawdata = 'kaggle/IA2-dev.csv'
devdata, dev_id = preprocess(rawdata, donormalize=donormalize, istrain=0,
                             traininfo=traindata, doengr=doengr)

# Test
rawdata = 'kaggle/IA2-test.csv'
testdata, test_id = preprocess(rawdata, donormalize=donormalize, istrain=0,
                             traininfo=traindata, doengr=doengr)

data size (rows,columns) (16000, 198)
                    Vehicle_Damage  Previously_Insured  Vehicle_Age_0  \
Vehicle_Damage            1.000000           -0.855336       0.309945   
Previously_Insured       -0.855336            1.000000      -0.309128   
Vehicle_Age_0             0.309945           -0.309128       1.000000   
Vehicle_Age_1            -0.420687            0.412486      -0.859129   
Vehicle_Age_2             0.177374           -0.163885      -0.342348   
Response                  0.599459           -0.587338       0.257524   

                    Vehicle_Age_1  Vehicle_Age_2  Response  
Vehicle_Damage          -0.420687       0.177374  0.599459  
Previously_Insured       0.412486      -0.163885 -0.587338  
Vehicle_Age_0           -0.859129      -0.342348  0.257524  
Vehicle_Age_1            1.000000      -0.186714 -0.342083  
Vehicle_Age_2           -0.186714       1.000000  0.133690  
Response                -0.342083       0.133690  1.000000  

data size (rows,column

In [3]:
# DEV: Model Training and Selection

# - max. number of iterations (fixed) - epochs
epochs = int(9e3) # early-stoppng is key here

# - learning-rate (step-size) selection set
# lrs =  [5e-3, 1e-2, 2e-2, 0.1, 0.5]
# learning rate
stepsize = 1e-1

# - regularization scale size selection set
# lregs = [1e-3, 2e-3, 5e-3, 1e-2] # ... 1e-1, 1, 1e1, 1e2, 1e3]
lregs = [0.00055] #[1e-4, 5e-4, 1e-3]
# regsize = 0.01 # 1e-1 to 1e-2 to 1e-3

# - random weight initialization
W = np.random.uniform(0, 0.01, (traindata['cols'], 1))
W[0] = np.zeros(shape=(1, 1))

# Turns out lists and dicts are passed by ref. in python.
# They behave as global variables, modified in function they are passed to.

# model's number of largest weighted features
bigks = 10

# list to hold all models
model_sels = []

# regularization rate
# regtype = 1
regtype = 2
# regtype = 3
# regtype = 0

# for stepsize in lrs:
for regsize in lregs:

    print(f'\n*******L{regtype}: Model Selection************************')
    print('(start): Regularization-scale: ', regsize)
    # print(W.T) # to debug muatbility

    # - modeldict: data structure that holds details of the trained model
    modeldict = {'W': W.copy(), 'stepsize': stepsize,
                 'reg_type': regtype, 'reg_size': regsize, 'epochs': epochs,
                 'cols': traindata['cols'],
                 'normalize': traindata['scalers'],
                 'mse_train': None, 'mse_dev': None,
                 'facc_train': None, 'facc_dev': None,
                 'sparsity': None, 'bigk': bigks, 'WBigK': None, 'WBigK_feats': None
                 }

    # - train: iterative line search (full batch)
    batchgd(modeldict, stdlogistic, traindata, devdata)

    model_sels.append(modeldict)

    if np.isfinite(modeldict['mse_train'][-1]):
        print(f"MSE (Train): {modeldict['mse_train'][-1]:2.4f} | "
              f"(Validation): {modeldict['mse_dev'][-1]:2.4f}")
        print(f"Class Accuracy (Train): {modeldict['facc_train'][-1]:2.4f} | "
              f"(Validation): {modeldict['facc_dev'][-1]:2.4f}")

        # print(W.T)
        print(f"Model Sparsity: {modeldict['sparsity']}")
        print('Final Learned Weights')
        # print((modeldict['W']).T)
        # Print top 5 weighted features
        print("Features (Top-5) with largest weight magnitude")
        print(modeldict['WBigK_feats'].tolist())
        wbigk_str = np.array2string(modeldict['WBigK'].flatten(),
                                    formatter={'float_kind': '{0:2.4f}'.format})
        print(wbigk_str)
        print('(end): ----\n')




*******L2: Model Selection************************
(start): Regularization-scale:  0.00055
k:     0, mse(train): 0.2501, mse(dev): 0.2416 | facc(train): 0.4941, facc(dev): 0.7316
k:   500, mse(train): 0.1464, mse(dev): 0.1462 | facc(train): 0.7982, facc(dev): 0.8008
k:  1000, mse(train): 0.1453, mse(dev): 0.1451 | facc(train): 0.7981, facc(dev): 0.7997
k:  1500, mse(train): 0.1448, mse(dev): 0.1447 | facc(train): 0.7984, facc(dev): 0.8000
k:  2000, mse(train): 0.1446, mse(dev): 0.1444 | facc(train): 0.7984, facc(dev): 0.7999
k:  2500, mse(train): 0.1444, mse(dev): 0.1442 | facc(train): 0.7985, facc(dev): 0.8000
k:  3000, mse(train): 0.1443, mse(dev): 0.1441 | facc(train): 0.7986, facc(dev): 0.7999
k:  3500, mse(train): 0.1442, mse(dev): 0.1439 | facc(train): 0.7984, facc(dev): 0.7998
k:  4000, mse(train): 0.1441, mse(dev): 0.1439 | facc(train): 0.7985, facc(dev): 0.8000
k:  4500, mse(train): 0.1441, mse(dev): 0.1438 | facc(train): 0.7986, facc(dev): 0.8001
k:  5000, mse(train): 0.1440

In [4]:
# Test prediction INFERENCE
from src.infer_models import infer
from src.archs_models import stdlogistic

for mdl in model_sels:
  Yhat_test = infer(stdlogistic, mdl, testdata)
  test_result = {'ID': test_id, 'Response': Yhat_test.flatten()}
  dftest = pd.DataFrame(data=test_result)
  dftest.to_csv(f"kaggle/PA2_preds_oas_L{mdl['reg_type']}_{mdl['reg_size']}_{mdl['epochs']}.csv", index=False)


# -- gender kept 10000 -> gave higher train accuracy
# -- gender not kept 10000 -> gave higher train accuracy than gender kept with 1e-4 > 1e-3
